In [1]:
%reload_ext sql
%config SqlMagic.autocommit=False
%config SqlMagic.autolimit=0
%config SqlMagic.autopandas=True
%config SqlMagic.displaylimit=200

In [2]:
%sql trino://localhost:9090/cuebiq/

'Connected: @cuebiq/'

In [3]:
pip install python-geohash

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import geohash
from datetime import datetime, timedelta
import logging
from sqlalchemy import create_engine
import time

In [5]:
# SQL engine
from trino.dbapi import connect 
from sqlalchemy import create_engine
import pandas as pd
import time

class TrinoEngine():
    def __init__(self):
        conn = connect(
            host="localhost",
            port=9090,
            catalog="cuebiq"
        )
        self.cur = conn.cursor()
        self.engine = create_engine("trino://localhost:9090/cuebiq/")
    
    def execute_statement(self, query:str) -> list:
        """
        Create and drop statements.
        """
        self.cur.execute(query)
        return self.cur.fetchall()
    
    def read_sql(self, query:str) -> pd.DataFrame: 
        """
        Select and insert into operations.
        """
        return pd.read_sql(query, self.engine)

sql_engine = TrinoEngine()

In [6]:
schema_name = {'cda': 'cuebiq.paas_cda_pe_v3'}
pe_tj_table = f"{schema_name['cda']}.trajectory_uplevelled"

In [7]:
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Database connection setup
output_schema_name = 'od_matrix_10'
con = create_engine(f"trino://localhost:9090/dedicated/{output_schema_name}")

In [8]:
# Function to process data for a single day
def process_day(event_date, country_code, sql_engine):
    try:
        # Read data from the SQL table
        pe_tj_df = sql_engine.read_sql(
            f"""
            SELECT 
                cuebiq_id,
                start_lat,
                start_lng,
                end_lat,
                end_lng,
                duration_minutes,
                length_meters,
                number_of_points
            FROM cuebiq.paas_cda_pe_v3.trajectory_uplevelled
            WHERE 
                event_date = {event_date}
                AND end_country = '{country_code}' 
                AND start_country = '{country_code}' 
            """
        )
        logging.info(f"Executing SQL query for date {event_date}")

        # Encode geohashes
        pe_tj_df['start_geohash3'] = pe_tj_df.apply(
            lambda x: geohash.encode(x['start_lat'], x['start_lng'], precision=3), axis=1)
        pe_tj_df['end_geohash3'] = pe_tj_df.apply(
            lambda x: geohash.encode(x['end_lat'], x['end_lng'], precision=3), axis=1)

        # Add user numbers to the aggregated data
        aggregated_df3 = pe_tj_df.groupby(['start_geohash3', 'end_geohash3']).agg({
            'cuebiq_id': 'count',
            'duration_minutes': ['mean', 'median', 'std'],
            'length_meters': ['mean', 'median', 'std'],
            'number_of_points': ['mean', 'median', 'std']
        }).reset_index()
        aggregated_df3.columns = ['start_geohash3', 'end_geohash3', 'trip_count', 
                                  'm_duration_min', 'mdn_duration_min', 'sd_duration_min', 
                                  'm_length_m', 'mdn_length_m', 'sd_length_m', 
                                  'm_points_no', 'mdn_points_no', 'sd_points_no']

        # Filter aggregated data and reorder columns
        filtered_df3= aggregated_df3.loc[aggregated_df3['trip_count'] > 9]
        filtered_df3 = filtered_df3[['start_geohash3', 'end_geohash3', 'trip_count', 
                                     'm_duration_min', 'mdn_duration_min', 'sd_duration_min', 
                                     'm_length_m', 'mdn_length_m', 'sd_length_m', 
                                   'm_points_no', 'mdn_points_no', 'sd_points_no']]
        return filtered_df3

    except Exception as e:
        logging.error(f"Error processing data for date {event_date}: {e}")
        return pd.DataFrame()

# Function to insert data with retry mechanism
def insert_data_with_retry(df, table_name, con, retries=3, delay=5):
    for attempt in range(retries):
        try:
            df.to_sql(
                table_name, 
                con, 
                index=False, 
                if_exists="append", 
                method="multi"
            )
            logging.info(f"Inserted data into table {table_name}")
            break
        except Exception as e:
            logging.error(f"Attempt {attempt+1} failed with error: {e}")
            if attempt < retries - 1:
                time.sleep(delay)
            else:
                logging.error(f"Failed to insert data into table {table_name} after {retries} attempts")

# Main processing loop
def process_date_range(start_date, end_date, country_code, sql_engine):
    start_time = time.time()  # Record start time before processing loop
        
    current_date = start_date
    while current_date <= end_date:
        event_date = current_date.strftime('%Y%m%d')
        
        filtered_df3 = process_day(event_date, country_code, sql_engine)

        # Insert data into the database
        if not filtered_df3.empty:
            # table_name = f"{output_schema_name}.{country_code}_agg3"
            table_name = f"od_{country_code.lower()}_{event_date}_agg3_10"
            insert_data_with_retry(filtered_df3, table_name, con)
        else:
            logging.info(f"No data to insert for date {event_date}")

        # Move to the next day
        current_date += timedelta(days=1)
    end_time = time.time()  # Record end time after processing loop
    total_time = end_time - start_time
    logging.info(f"Total processing time: {total_time:.2f} seconds")

In [9]:
# Example usage:
process_date_range(datetime(2019, 12, 31), datetime(2019, 12, 31), 'IN', sql_engine)

2024-06-20 17:57:41,837 - INFO - Executing SQL query for date 20191231
2024-06-20 17:57:41,850 - INFO - No data to insert for date 20191231
2024-06-20 17:57:41,851 - INFO - Total processing time: 17.00 seconds


In [9]:
# # Check single data

# country_code = 'IN'

# pe_tj_df = sql_engine.read_sql(
#     f"""
#     SELECT 
#         cuebiq_id,
#         start_lat,
#         start_lng,
#         end_lat,
#         end_lng,
#         duration_minutes,
#         length_meters,
#         number_of_points
#     FROM cuebiq.paas_cda_pe_v3.trajectory_uplevelled
#     WHERE 
#         event_date = 20191231
#         AND end_country = '{country_code}' 
#         AND start_country = '{country_code}' 
#     """
# )
# pe_tj_df

,cuebiq_id,start_lat,start_lng,end_lat,end_lng,duration_minutes,length_meters,number_of_points
0,2134531595,28.555631,77.083816,28.555225,77.085207,37.533333,211.626826,4
1,2134531595,28.555225,77.085207,28.554646,77.085500,8.416667,70.557999,2
2,2134531595,28.554646,77.085500,28.552414,77.087340,22.616667,399.600430,3
3,2134531595,28.552414,77.087340,28.553300,77.084860,11.033333,261.781491,2
4,2134531595,28.553300,77.084860,28.552529,77.087346,5.983333,257.788970,2
5,2568416615,19.098772,72.874246,19.098587,72.873595,12.050000,244.820889,4
6,2568416615,19.098587,72.873595,19.098606,72.873460,13.016667,245.590944,4
7,2482705859,28.555338,77.083773,28.554896,77.087750,29.633333,952.883394,9
8,2482705859,28.554896,77.087750,28.670810,77.186134,35.366667,17913.335777,7
9,2042706835,28.554834,77.085579,28.563946,77.084399,316.316667,1798.265225,10
